In [1]:
import random
import time
from IPython.display import clear_output

In [2]:
def combinacoes_de_dados(lista):
    combinacoes = set()

    combinacoes.add((lista[0] + lista[1],lista[2] + lista[3]))
    combinacoes.add((lista[0] + lista[2],lista[1] + lista[3]))
    combinacoes.add((lista[0] + lista[3],lista[1] + lista[2]))

    return list(combinacoes)


In [3]:
class Column():
    def __init__(self,length):
        self.length=length
        self.position={1:0,2:0}
        self.marker=0
        self.blocked=False

    def advance_marker(self):
        self.marker+=1

    def record_advance(self,active_player):
        self.position[active_player] = self.marker
        self.marker_reset()

    def marker_reset(self):
        self.marker = 0

    def marker_start(self,active_player):
        self.marker = self.position[active_player] + 1

    def possui_marcador(self):
        return self.marker > 0
    
    def is_blocked(self):
        return self.blocked

In [4]:
n_colunas = [2,4,6,8,10,12,10,8,6,4,2]
class Board():
    def __init__(self):
        self.columns={}
        for i in range(11):
            self.columns[i+2] = Column(n_colunas[i])
    
    def roll_dice(self):
        return combinacoes_de_dados([random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7)])

    def lista_colunas_com_marcadores(self):
        return list(filter( lambda coluna : self.columns[coluna].possui_marcador(), self.columns))
    
    def lista_colunas_bloqueadas(self):
        return list(filter( lambda coluna : self.columns[coluna].is_blocked(), self.columns))
       

In [5]:
class Game_settings():
    def __init__(self):
        self.marker_count=3
    
    def set_rules(self):
        global marker_count
        marker_count=self.marker_count

In [34]:
def run_game():
    #initialize the classes
    board=Board()
    active_player=1
    #choose your move
    def make_move(your_choice):
        '''execute the chosen move'''
        #make your move
        while(your_choice!=[]):
            
            for i in range (2,13):
                if your_choice==[]:
                    break
                elif (your_choice[0] not in board.lista_colunas_com_marcadores()) and len(board.lista_colunas_com_marcadores())<marker_count: #se tem marcardor, bota marcador

                    board.columns[your_choice[0]].marker_start(active_player)
                    print(f'placed marker at column: {your_choice[0]}')
                    print(your_choice[0], board.columns[your_choice[0]].position)
                    print(your_choice[0], board.columns[your_choice[0]].marker)
                    your_choice.remove(your_choice[0])

                elif your_choice[0] in board.lista_colunas_com_marcadores(): #se tem marcador, avança marcador

                    board.columns[your_choice[0]].advance_marker()
                    print(f'advanced marker at column: {your_choice[0]}')
                    print(your_choice[0], board.columns[your_choice[0]].position)
                    print(your_choice[0], board.columns[your_choice[0]].marker)
                    your_choice.remove(your_choice[0])
                else:
                    for i in range (2,13):
                        board.columns[i].record_advance(active_player)
                        print(f'marked advance on column {i}')
    def pass_turn(active_player):
        '''voluntarily end your turn'''
        print(f'player: {active_player} records their markers positions and passes their turn')
        for i in range (2,13):
            if i in board.lista_colunas_com_marcadores():
                board.columns[i].record_advance(active_player)

            
    for i in range(50):
        
        time.sleep(0.1)
        clear_output(wait=True)
        your_dice=board.roll_dice()
        your_choice=list(your_dice[random.randrange(0,len(your_dice))])
        print(your_choice)
        make_move(your_choice)
        pass_turn(active_player)
        if active_player==1:
            active_player=2
        else:
            active_player=1

In [7]:
class Player():
    def __init__(self,name):
        player.name=self.name


In [8]:
game_settings=Game_settings()
game_settings.set_rules()

In [40]:
def run_game():
    #initialize the classes
    board=Board()
    active_player=1
    #choose your move
    def make_move(your_choice):
        '''execute the chosen move'''
        #make your move
        while(your_choice!=[]):
            
            for i in range (2,13):
                if your_choice==[]:
                    break
                elif (your_choice[0] not in board.lista_colunas_com_marcadores()) and len(board.lista_colunas_com_marcadores())<marker_count: #se tem marcardor, bota marcador

                    board.columns[your_choice[0]].marker_start(active_player)
                    print(f'placed marker at column: {your_choice[0]}')
                    print(your_choice[0], board.columns[your_choice[0]].position)
                    print(your_choice[0], board.columns[your_choice[0]].marker)
                    your_choice.remove(your_choice[0])

                elif your_choice[0] in board.lista_colunas_com_marcadores(): #se tem marcador, avança marcador

                    board.columns[your_choice[0]].advance_marker()
                    print(f'advanced marker at column: {your_choice[0]}')
                    print(your_choice[0], board.columns[your_choice[0]].position)
                    print(your_choice[0], board.columns[your_choice[0]].marker)
                    your_choice.remove(your_choice[0])
                else:
                    for i in range (2,13):
                        board.columns[i].record_advance(active_player)
                        print(f'marked advance on column {i}')
    def pass_turn(active_player):
        '''voluntarily end your turn'''
        print(f'player: {active_player} records their markers positions and passes their turn')
        for i in range (2,13):
            if i in board.lista_colunas_com_marcadores():
                board.columns[i].record_advance(active_player)

            
    for i in range(30):
        
        time.sleep(0.1)
        clear_output(wait=True)
        your_dice=board.roll_dice()
        your_choice=list(your_dice[random.randrange(0,len(your_dice))])
        print(your_choice)
        make_move(your_choice)
        pass_turn(active_player)
        if active_player==1:
            active_player=2
        else:
            active_player=1

In [38]:
game1=Game_manager()
game1.run_game()

[5, 6]


NameError: name 'make_move' is not defined

In [41]:
run_game()

[9, 9]
placed marker at column: 9
9 {1: 3, 2: 1}
9 2
advanced marker at column: 9
9 {1: 3, 2: 1}
9 3
player: 2 records their markers positions and passes their turn


In [ ]:
for i in range (1000):
    clear_output(wait=True)
    print(i)


In [ ]:
a=board.columns[5].position
a

In [ ]:
board.roll_dice()

In [ ]:
your_dice=board.roll_dice()
your_choice=your_dice[random.randrange(0,len(your_dice))]

In [ ]:
your_choice=your_choice-your_choice[0]